In [ ]:
!hdfs dfs -ls /

In [9]:
!sudo hdfs dfs -mkdir -p /user/hadoop-repo


We trust you have received the usual lecture from the local System
Administrator. It usually boils down to these three things:

    #1) Respect the privacy of others.
    #2) Think before you type.
    #3) With great power comes great responsibility.

[sudo] password for alice: 


In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -put

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -put text hadoop-repo

In [ ]:
!hdfs --config ~/hadoop_palmetto/config fsck hadoop-repo/text/complete-shakespeare.txt -files -blocks -locations

In [ ]:
!mkdir codes

In [ ]:
!pwd

In [ ]:
!hdfs dfs -cat hadoop-repo/text/complete-shakespeare.txt \
    2>/dev/null | head -n 100

In [ ]:
%%writefile codes/wordcountMapper.py

#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python   

import sys                                                                                                
for oneLine in sys.stdin:
    oneLine = oneLine.strip()
    for word in oneLine.split(" "):
        if word != "":
            print ('%s\t%s' % (word, 1))
        

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -cat hadoop-repo/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py
    | sort


In [ ]:
%%writefile codes/wordcountReducer.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python
import sys

current_word = None
total_word_count = 0

for line in sys.stdin:
    line = line.strip()
    word, count = line.split("\t", 1)
    try:
        count = int(count)
    except ValueError:
        continue
    
    if current_word == word:
        total_word_count += count
    else:
        if current_word:
            print ("%s\t%s" % (current_word, total_word_count))
        current_word = word
        total_word_count = 1
        
if current_word == word:
    print ("%s\t%s" % (current_word, total_word_count))

In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -cat hadoop-repo/text/complete-shakespeare.txt \
    2>/dev/null \
    | head -n 20 \
    | python ./codes/wordcountMapper.py
    | sort \
        | python ./codes/wordcountReducer.py


In [ ]:
!hdfs --config ~/hadoop_palmetto/config dfs -rm -R hadoop-repo/output-wordcount

!mapred --config ~/hadoop_palmetto/config streaming \
    -input intro-to-hadoop/text/complete-shakespeare.txt \
    -output hadoop-repo/output-wordcount \
    -file ./codes/wordcountMapper.py \
    -mapper wordcountMapper.py \
    -file ./codes/wordcountReducer.py \
    -reducer wordcountReducer.py

In [ ]:
!hdfs dfs -ls hadoop-repo/output-wordcount

In [ ]:
!hdfs dfs -cat hadoop-repo/output-wordcount/part-00000 \
    2>/dev/null | head -n 100

In [ ]:
%%writefile codes/wordcountEnhancedMapper.py
#!/software/spackages/linux-centos8-x86_64/gcc-8.3.1/anaconda3-2019.10-v5cuhr6keyz5ryxcwvv2jkzfj2gwrj4a/bin/python                                          
import sys                     
import string

translator = str.maketrans('', '', string.punctuation)

for oneLine in sys.stdin:
    oneLine = oneLine.strip()
    for word in oneLine.split(" "):
        if word != "":
            newWord = word.translate(translator).lower()
            print ('%s\t%s' % (_______, 1))

In [ ]:
!hdfs dfs -rm -R hadoop-repo/output-wordcount-enhanced

!ssh dsciutil yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input hadoop-repo/text/gutenberg-shakespeare.txt \
    -output hadoop-repo/output-wordcount \
    -file ____________________________________________________ \
    -mapper _____________________ \
    -file ____________________________________________________ \
    -reducer _____________________ \

In [ ]:
Debug jobs

In [ ]:
Dataset
http://files.grouplens.org/datasets/movielens/ml-10m-README.html

In [ ]:
!hdfs dfs -ls -h /repository/movielens

In [ ]:
Find movies which have the highest average ratings over the years and report their ratings and genres
Find the average ratings of all movies over the years
Sort the average ratings from highest to lowest
Report the results, augmented by genres

In [ ]:
!hdfs dfs -ls /repository/movielens
!hdfs dfs -cat /repository/movielens/README.txt

!hdfs dfs -cat /repository/movielens/links.csv \
    2>/dev/null | head -n 5

!hdfs dfs -cat /repository/movielens/movies.csv \
    2>/dev/null | head -n 5

!hdfs dfs -cat /repository/movielens/ratings.csv \
    2>/dev/null | head -n 5

!hdfs dfs -cat /repository/movielens/tags.csv \
    2>/dev/null | head -n 5

In [ ]:
%%writefile codes/avgRatingMapper.py
#!/usr/bin/env python

import sys
import csv 

movieFile = "./movielens/movies.csv"
movieList = {}


with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2]

for oneMovie in sys.stdin:
    oneMovie = oneMovie.strip()
    ratingInfo = oneMovie.split(",")
    try:
        movieTitle = movieList[ratingInfo[1]]["title"]
        movieGenre = movieList[ratingInfo[1]]["genre"]
        rating = float(ratingInfo[2])
        print ("%s\t%s\t%s" % (movieTitle, rating, movieGenre))
    except ValueError:
        continue

In [ ]:
!hdfs dfs -cat /repository/movielens/ratings.csv \
    2>/dev/null | head -n 10 | python ./codes/avgRatingMapper.py

In [ ]:
!mkdir movielens
!mkdir movielens
!hdfs dfs -get /repository/movielens/movies.csv movielens/movies.csv


In [ ]:
# Test Reducer 
%%writefile codes/avgRatingReducer.py
#!/usr/bin/env python
import sys

current_movie = None
current_rating_sum = 0
current_rating_count = 0

for line in sys.stdin:
    line = line.strip()
    movie, rating, genre = line.split("\t", 2)
    try:
        rating = float(rating)
    except ValueError:
        continue

    if current_movie == movie:
        current_rating_sum += rating
        current_rating_count += 1
    else:
        if current_movie:
            rating_average = current_rating_sum / current_rating_count
            print ("%s\t%s\t%s" % (current_movie, rating_average, genre))    
        current_movie = movie
        current_rating_sum = rating
        current_rating_count = 1

if current_movie == movie:
    rating_average = current_rating_sum / current_rating_count
    print ("%s\t%s\t%s" % (current_movie, rating_average, genre))




In [ ]:
!hdfs dfs -cat /repository/movielens/ratings.csv 2>/dev/null \
    | head -n 5 \
    | python ./codes/avgRatingMapper.py \
    | sort \
    | python ./codes/avgRatingReducer.py

In [ ]:
# HDFS execution
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input /repository/movielens/ratings.csv \
    -output hadoop-repo/output-movielens-01 \
    -file ./codes/avgRatingMapper.py \
    -mapper avgRatingMapper.py \
    -file ./codes/avgRatingReducer.py \
    -reducer avgRatingReducer.py \

In [ ]:
# Log YARN 
! yarn logs -applicationId APPLICATION_ID


In [ ]:
#!yarn logs -applicationId application_1476193845089_0123 (tail -l 500)
stderr: Error messages from the actual task execution
stdout: Print out messages if the task includes them
syslog: Logging messages from the Hadoop MapReduce operation

In [ ]:
!yarn logs -applicationId application_1505269880969_0056 | grep -v INFO

!yarn logs -applicationId APPLICATION_ID | grep '^Container:'

!yarn logs -applicationId application_1505269880969_0056 | grep '^Container:'

In [ ]:
!yarn logs -applicationId APPLICATION_ID -containerId CONTAINER_ID --nodeAddress NODE_ADDRESS \
    | grep -v DEBUG

In [ ]:
# AvgMap

In [ ]:
%%writefile codes/avgRatingMapper02.py
#!/usr/bin/env python

import sys
import csv

movieFile = "./movies.csv"
movieList = {}


with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2]

for oneMovie in sys.stdin:
    oneMovie = oneMovie.strip()
    ratingInfo = oneMovie.split(",")
    try:
        movieTitle = movieList[ratingInfo[1]]["title"]
        movieGenre = movieList[ratingInfo[1]]["genre"]
        rating = float(ratingInfo[2])
        if _________:
            print ("%s\t%s\t%s" % (movieTitle, rating, movieGenre))
    except ValueError:
        continue

In [ ]:
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input /repository/movielens/ratings.csv \
    -output hadoop-repo/output-movielens-challenge \
    -file ____________ \
    -mapper ___________ \
    -file ./codes/avgRatingReducer01.py \
    -reducer avgRatingReducer01.py \
    -file ./codes/movielens/movies.csv

In [ ]:
Optimization


In [ ]:
# Reduce data movement in the shuffle phase

!hdfs dfs -rm -r hadoop-repo/output-movielens-02

!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input /repository/movielens/ratings.csv \
    -output hadoop-repo/output-movielens-02 \
    -file ./codes/avgRatingMapper.py \
    -mapper avgRatingMapper.py \
    -file ./codes/avgRatingReducer.py \
    -reducer avgRatingReducer.py \
    -file ./movielens/movies.csv

In [ ]:
%%writefile codes/avgRatingReducer-exp.py
#!/usr/bin/env python
import sys
import csv

movieFile = "./movies.csv"
movieList = {}

with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2]

current_movie = None
current_rating_sum = 0
current_rating_count = 0

for line in sys.stdin:
    line = line.strip()
    movie, rating = line.split("\t", 1)
    try:
        rating = float(rating)
    except ValueError:
        continue

    if current_movie == movie:
        current_rating_sum += rating
        current_rating_count += 1
    else:
        if current_movie:
            rating_average = current_rating_sum / current_rating_count
            movieTitle = movieList[current_movie]["title"]
            movieGenres = movieList[current_movie]["genre"]
            print ("%s\t%s\t%s" % (movieTitle, rating_average, movieGenres))    
        current_movie = movie
        current_rating_sum = rating
        current_rating_count = 1

if current_movie == movie:
    rating_average = current_rating_sum / current_rating_count
    movieTitle = movieList[current_movie]["title"]
    movieGenres = movieList[current_movie]["genre"]
    print ("%s\t%s\t%s" % (movieTitle, rating_average, movieGenres))


In [ ]:
!hdfs dfs -rm -r hadoop-repo/output-movielens-03

!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input /repository/movielens/ratings.csv \
    -output hadoop-repo/output-movielens-03 \
    -file ./codes/avgRatingMapper-exp.py \
    -mapper avgRatingMapper-exp.py \
    -file ./codes/avgRatingReducer.py \
    -reducer avgRatingReducer.py \
    -file ./movielens/movies.csv

In [ ]:
# Check output dir
!hdfs dfs -ls hadoop-repo/output-movielens-02
!hdfs dfs -ls hadoop-repo/output-movielens-03

!hdfs dfs -cat hadoop-repo/output-movielens-03/part-00000 \
    2>/dev/null | head -n 20

In [ ]:
# Find genres which have the highest average ratings over the years
# Common optimization approaches:

# In-mapper reduction of key/value pairs
# Additional combiner function
%%writefile codes/avgGenreMapper.py
#!/usr/bin/env python
import sys
import csv

# for nonHDFS run
movieFile = "./movielens/movies.csv"

# for HDFS run
#movieFile = "./movies.csv"
movieList = {}

with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2]

for oneMovie in sys.stdin:
    oneMovie = oneMovie.strip()
    ratingInfo = oneMovie.split(",")
    try:
        genreList = movieList[ratingInfo[1]]["genre"]
        rating = float(ratingInfo[2])
        for genre in genreList.split("|"):
            print ("%s\t%s" % (genre, rating))
    except ValueError:
        continue


In [ ]:
# Optimization through in-mapper reduction of Key/Value pairs
%%writefile codes/avgGenreMapper-comb.py
#!/usr/bin/env python
import sys
import csv

# for nonHDFS run
movieFile = "./movielens/movies.csv"

# for HDFS run
#movieFile = "./movies.csv"
movieList = {}

with open(movieFile, mode = 'r') as infile:
    reader = csv.reader(infile)
    for row in reader:
        movieList[row[0]] = {}
        movieList[row[0]]["title"] = row[1]
        movieList[row[0]]["genre"] = row[2]

for oneMovie in sys.stdin:
    oneMovie = oneMovie.strip()
    ratingInfo = oneMovie.split(",")
    try:
        genreList = movieList[ratingInfo[1]]["genre"]
        rating = float(ratingInfo[2])
        for genre in genreList.split("|"):
            print ("%s\t%s" % (genre, rating))
    except ValueError:
        continue


In [ ]:
# 
%%writefile codes/avgGenreReducer-comb.py
#!/usr/bin/env python
import sys
import csv
import json

current_genre = None
current_rating_sum = 0
current_rating_count = 0

for line in sys.stdin:
    line = line.strip()
    genre, rating = line.split("\t", 1)

    if current_genre == genre:
        try:
            current_rating_sum += float(rating)
            current_rating_count += 1
        except ValueError:
            continue    
    else:
        if current_genre:
            rating_average = current_rating_sum / current_rating_count
            print ("%s\t%s" % (current_genre, rating_average))    
        current_genre = genre
        try:
            current_rating_sum = float(rating)
            current_rating_count = 1
        except ValueError:
            continue

if current_genre == genre:
    rating_average = current_rating_sum / current_rating_count
    print ("%s\t%s" % (current_genre, rating_average))

In [ ]:
# !hdfs dfs -cat hadoop-repo/output-movielens-04/part-00000

In [ ]:
!hdfs dfs -cat /repository/movielens/ratings.csv 2>/dev/null \
    | head -n 10 \
    | python ./codes/avgGenreMapper-comb.py \

In [ ]:
# !hdfs dfs -cat /repository/movielens/ratings.csv 2>/dev/null \
#     | head -n 10 \
#     | python ./codes/avgGenreMapper02.py \
#     | sort \
#     | python ./codes/avgGenreReducer02.py

In [ ]:
%%writefile codes/avgGenreCombiner.py
#!/usr/bin/env python

import sys
import csv
import json

genreList = {}

for line in sys.stdin:
    line = line.strip()
    genre, ratingString = line.split("\t", 1)
    ratingInfo = json.loads(ratingString)

    if genre in genreList:
        genreList[genre]["total_rating"] += ratingInfo["total_rating"]
        genreList[genre]["total_count"] += ratingInfo["total_count"]
    else:
        genreList[genre] = {}
        genreList[genre]["total_rating"] = ratingInfo["total_rating"]
        genreList[genre]["total_count"] = 1

for genre in genreList:
    print ("%s\t%s" % (genre, json.dumps(genreList[genre])))


In [ ]:
!yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input /repository/movielens/ratings.csv \
    -output hadoop-repo/output-movielens-06 \
    -file ./codes/avgGenreMapper02.py \
    -mapper avgGenreMapper02.py \
    -file ./codes/avgGenreReducer02.py \
    -reducer avgGenreReducer02.py \
    -file ./codes/avgGenreCombiner.py \
    -combiner avgGenreCombiner.py \
    -file ./movielens/movies.csv

In [ ]:
Integrating Hadoop job into workflow HDP workflow

In [ ]:
%%writefile movieAnalyzer.pbs
#!/bin/bash

#PBS -N movieData
#PBS -l select=1:ncpus=8:mem=8gb
#PBS -l walltime=00:15:00
#PBS -j oe

# load hdp module and initilalize Keberos tokens
module load hdp/0.1
cypress-kinit
klist

# cd into directory containing the PBS script
cd $PBS_O_WORKDIR

# attempt to remove output directory
hdfs dfs -rm -r hadoop-repo/output-movielens-03

# 
# submit Hadoop job to Cypress
yarn jar /usr/hdp/current/hadoop-mapreduce-client/hadoop-streaming.jar \
    -input /repository/movielens/ratings.csv \
    -output hadoop-repo/output-movielens-03 \
    -file ./codes/avgRatingMapper01.py \
    -mapper avgRatingMapper01.py \
    -file ./codes/avgRatingReducer.py \
    -reducer avgRatingReducer.py \
    -file ./movielens/movies.csv

# export output data back to Palmetto for further analysis
hdfs dfs -get hadoop-repo/output-movielens-03/part-00000 .

In [ ]:
# View final output when job is finished

ssh login001
cd ~/hadoop-repo
qsub movieAnalyzer.pbs


!qstat -anu $USER
!cat part-00000 2>/dev/null | head -n 20

In [ ]:
# CLean up 
!hdfs dfs -ls hadoop-repo
!hdfs dfs -rm -r hadoop-repo/
!rm -Rf codes/
!rm -Rf movielens/